# ***Tesseract***

In [ ]:
!pip install PyPDF2

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/232.6 kB 186.2 kB/s eta 0:00:02
   ----- --------------------------------- 30.7/232.6 kB 186.2 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/232.6 kB 178.6 kB/s eta 0:00:02
   ---------- ---------------------------- 61.4/232.6 kB 233.8 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/232.6 kB 233.8 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/232.6 kB 233.8 kB/s eta 0:00:01
   ---------- -----------------------

In [ ]:
!pip install gradio

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/67/48/d71c7bbb92e72b30921cdb7034a437930695e7d7fd83290b766974792d31/gradio-4.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for aiofiles<24.0,>=22.0 from https://files.pythonhosted.org/packages/c5/19/5af6804c4cc0fed83f47bff6e413a98a36618e7d40185cd36e69737f3b0e/aiofiles-23.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/c0/c1/2dc286475c8e2e455e431a1cf1cf29662c9f9290434161088ba039d77481/fastapi-0.110.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for gradio-client==0.15.1 from https://files.pythonhosted.org/packages/12/99/433f90d17d24738ed795b7c9b78ba1e4149b97a8dfe927e2f6e5f537d541/gradio_client-0.15.1-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-resources<7.0,>=

In [ ]:
import gradio as gr
import PyPDF2
import numpy as np

# Gradio interface creation
interface = gr.Interface(
    fn=extract,
    inputs="file",  # Specify input type
    outputs="text",
    elem_id="upload",  # Assign an element ID for customization (optional)
    title="PDF Text Classification",  # Use 'title' instead of 'label'
    description="Upload a PDF file to extract text and get its classification.",
)





In [ ]:
# Customize output (optional)
def format_output(outputs):
    if not outputs:
        return "No text found in the PDF."
    output_text = ""
    for text, class_name, class_prob in outputs:
        output_text += f"Class: {class_name} \n Text: {text}\n\n\n"
    return output_text

In [ ]:
demo = gr.Interface(fn=extract, inputs="file", outputs="text")
demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


incorrect startxref pointer(3)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image
from ultralytics.utils.plotting import Annotator, colors
import cv2
import os
import pytesseract
import pandas as pd

In [ ]:
model = YOLO('yolov8_detect_4classes_86map.pt')
names = model.names
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
names = model.names

In [ ]:
def extract(path):
    text = []
    clas = []
    im = []
    results = model(path)
    #img = cv2.imread(path)

    # Extract bounding boxes
    boxes = results[0].boxes.xyxy.tolist()
    clss = results[0].boxes.cls.cpu().tolist()
    #annotator = Annotator(img, line_width=2, example=names)

    # Iterate through the bounding boxes
    for box, cls in zip(boxes, clss):
        x1, y1, x2, y2 = box
        # Crop the object using the bounding box coordinates
        ultralytics_crop_object = path[int(y1):int(y2), int(x1):int(x2)]

        im.append(ultralytics_crop_object)

        #idx += 1
        #annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

            # Adding custom options
        custom_config = r'--oem 3 --psm 6'
        text.append(pytesseract.image_to_string(ultralytics_crop_object, config=custom_config))
        clas.append(names[int(cls)])

    return text , clas , im


In [ ]:
import PIL
import fitz
import numpy as np
import pandas as pd

def xx(file_path) :
    doc = fitz.open(file_path)  # open document
    text = []
    classes = []
    im = []
    for i, page in enumerate(doc):
        pix = page.get_pixmap()  # render page to an image
        #img = cv2.imread(pix)
        #pix.save(f"page_{i}.png")
        pix = PIL.Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        pix = np.array(pix)

        t , c , i = extract(pix)
        text.append(t)
        classes.append(c)
        im.append(i)

    text = [item for sublist in text for item in sublist]
    classes = [item for sublist in classes for item in sublist]
    im = [item for sublist in im for item in sublist]
    txt = ""
    for i in len(text) :
        txt+=f"Section : { classes } \n text : {text[i] } \n\n*************************************************************"
    return txt

In [ ]:
import PIL
import fitz
import numpy as np
import pandas as pd

file_path ="1705067515471.pdf"
doc = fitz.open(file_path)  # open document
text = []
classes = []
im = []
for i, page in enumerate(doc):
    pix = page.get_pixmap()  # render page to an image
    #img = cv2.imread(pix)
    #pix.save(f"page_{i}.png")
    pix = PIL.Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    pix = np.array(pix)

    t , c , i = extract(pix)
    text.append(t)
    classes.append(c)
    im.append(i)

text = [item for sublist in text for item in sublist]
classes = [item for sublist in classes for item in sublist]
im = [item for sublist in im for item in sublist]

df = pd.DataFrame({'Text': text, 'Class': classes})
df


    #cv2.show_image(pix)

0: 640x480 1 Experience, 2 skillss, 337.9ms
Speed: 5.1ms preprocess, 337.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 Certifications, 18.8ms
Speed: 2.0ms preprocess, 18.8ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 480)


,Text,Class
0,Professional Experience\n200307. Intern\n2023-...,Experience
1,Skills\n\nClerical suppor\nResearch\nPresentat...,skills
2,Software\n\nLinux\n\nTableau\n\nWeb development\n,skills
3,"Certifications\n2023-07 [Web development], [Te...",Certifications


In [ ]:
print(df['Text'][0])

Professional Experience
200307. Intern
2023-09 Web Development
+ Sorted and organized files, spreadsheets, and
report
+ Interacted with customers by phone, email, or
in-person to provide information,
+ Completed research, compiled data, updated
spreadsheets, and produced timely reports
+ Analyzed problems and worked with teams to
develop solutions
+ Facilitated successful completion of projects from
concept to launch.on web page of home page
202306. Intern
2023-07 Tableau
+ Sorted ond organized files, spreadsheets, and
report
+ Interacted with customers by phone, email, or
in-person to provide information,
202304 Intern
2025-05 Marketing Specialist
* Sorted and organized files, spreadsheets, and
report
+ Interacted with customers by phone, email, or
in-person to provide information,
+ Explored new technologies and approaches to
streamline processes



In [ ]:
import re
def classify_text(text):
    # Define patterns for each class
    experience_pattern = r'\b(?:work\s*experience|experience\s*professionnelle|career\s*summary|résumé\s*de\s*carrière|key\s*responsibilities|responsabilités\s*clés|key\s*achievements|réalisation\s*clés|projets? d\'intégration|stage|stagiaire|tâches?\s*effectuées?|tâches?\s*réalisées?)\b'
    projects_pattern = r'\b(?:projects|project\s*experience|projets|expérience\s*de\s*projet)\b'
    skills_pattern = r'\b(?:skills|skill\s*set|compétences|compétence|informatique)\b'
    certificate_pattern = r'\b(?:certificate|certifications|certificat|certifications)\b'


    # Match patterns in the text
    if re.search(experience_pattern, text, re.IGNORECASE):
        return "Experience"
    elif re.search(projects_pattern, text, re.IGNORECASE):
        return "Projects"
    elif re.search(skills_pattern, text, re.IGNORECASE):
        return "Skills"
    elif re.search(certificate_pattern, text, re.IGNORECASE):
        return "Certificate"
    else:
        return "Unknown"

def classify_resume(resume_texts):
    classified_texts = []
    for text in resume_texts:
        classification = classify_text(text)
        classified_texts.append((text, classification))
    return classified_texts

In [ ]:
text , clas = extract("C:/Users/trabe/Downloads/Lettings-Administrator-CV-1.png")
#print(classified_resume)
# Create a DataFrame from the classified texts
#df = pd.DataFrame(classified_resume, columns=["Text", "Class"])

#print(df)
print(text)
print(clas)


image 1/1 C:\Users\trabe\Downloads\Lettings-Administrator-CV-1.png: 640x480 1 Experience, 1 skills, 95.3ms
Speed: 4.6ms preprocess, 95.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 480)
['Career Summary\nFeb 2021 - Present ‘Austen Lettings, Birmingham\nLettings Administrator\nOutline\nWork in the Birmingham branch of a leading national letting agent, responsible for the\nadministration of up to 35 residential property rentals within a5 miles radius.\nKey Responsibilities\n+ Maintain client databases, send internal correspondence and compose emails\n‘+ Conduct tenant screenings, credit and reference checks, and income verification\n‘+ Prepare and execute tenancy agreements in accordance with UK housing regulations\n‘+ Regularly inspect rental properties to ensure they meet safety and maintenance standards\n‘+ Oversee the rent collection process, ensuring that rents are paid on time\n‘+ Build positive tenant relationships by promptly addressing inquiries, concerns, and

In [ ]:
classified_resume = extract("C:/Users/trabe/OneDrive/Bureau/Capture d'écran 2024-04-01 033041.png")
#print(classified_resume)
# Create a DataFrame from the classified texts
df = pd.DataFrame(classified_resume, columns=["Text", "Class"])

print(df)




image 1/1 C:\Users\trabe\OneDrive\Bureau\Capture d'cran 2024-04-01 033041.png: 640x480 1 section, 302.0ms
Speed: 38.7ms preprocess, 302.0ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 480)
                                                Text     Class
0  Academic projects\n\nLinguatand jan 2023 ave 2...  Projects


In [ ]:
classified_resume = extract("C:/Users/trabe/Downloads/1639748048896.jpg")
#print(classified_resume)
# Create a DataFrame from the classified texts
df = pd.DataFrame(classified_resume, columns=["Text", "Class"])

print(df)



image 1/1 C:\Users\trabe\Downloads\1639748048896.jpg: 640x480 3 sections, 15.5ms
Speed: 70.2ms preprocess, 15.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)
                                                Text        Class
0  EXPERIENCE PROFESSIONNELLE\n\n2021 (3 mois) - ...   Experience
1  COMPETENCES INFORMATIQUE\nBack office UMark\n\...       Skills
2  FORMATIONS ET DIPLOMES\n\n2019- Cetilicat, For...  Certificate


In [ ]:
classified_resume = extract("C:/Users/trabe/Downloads/1666966905196.jpg")
#print(classified_resume)
# Create a DataFrame from the classified texts
df = pd.DataFrame(classified_resume, columns=["Text", "Class"])

print(df)



image 1/1 C:\Users\trabe\Downloads\1666966905196.jpg: 640x480 3 sections, 45.4ms
Speed: 4.0ms preprocess, 45.4ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 480)
                                                Text       Class
0  Expériences professionnelles\n© Projet d'intég...  Experience
1  Compétences\n* Langage de programmation\nJAVA\...      Skills
2  Dipl6mes et Formations\n+ Licence a l'Institut...      Skills


In [ ]:
df['Text'][1]

'Compétences\n* Langage de programmation\nJAVA\nHTML 5 / CSS 3\nPHP\nPython\nJavaScript /TypeScript\nc\n* Frameworks\nAngular 12\nSymfony 5\nSpring Boot\nBootstrap 5\nExpress Js\n* Développement mobile\n‘Android Studio\nflutter\n* Base de données\nMySQL\nOracle\nMongoDB\n* Autres\nNode JS\nLinux\nGit/Github\nExcel\nWord\nAdobe XD\n'

In [ ]:
results = model("C:/Users/trabe/Downloads/1666966905196.jpg")  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen



image 1/1 C:\Users\trabe\Downloads\1666966905196.jpg: 640x480 3 sections, 167.0ms
Speed: 5.2ms preprocess, 167.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 480)


# ***easy OCR***

In [ ]:
import easyocr

In [ ]:
reader = easyocr.Reader(['en','fr'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
def extract1(path):
    text = []
    results = model(path)
    #img = cv2.imread(path)

    # Extract bounding boxes
    boxes = results[0].boxes.xyxy.tolist()

    # Iterate through the bounding boxes
    for i, box in enumerate(boxes):
        txt = ""
        x1, y1, x2, y2 = box
        # Crop the object using the bounding box coordinates
        ultralytics_crop_object = img[int(y1):int(y2), int(x1):int(x2)]


            # Adding custom options
        custom_config = r'--oem 3 --psm 6'
        #text.append(pytesseract.image_to_string(ultralytics_crop_object, config=custom_config))
        bounds = reader.readtext(ultralytics_crop_object)
        for res in bounds:
            det, conf = res[1], res[2]
            txt = txt + '\n' + det
        text.append(txt)
        #im = plt.imread(r'D:\OCR_different_approach\font_text_sizes.jpg')
        #plt.imshow(im)

    return classify_resume(text)
    #return classify_resume(text)





In [ ]:
t = extract1("C:/Users/trabe/Downloads/1639748048896.jpg")


image 1/1 C:\Users\trabe\Downloads\1639748048896.jpg: 640x480 3 sections, 176.9ms
Speed: 6.7ms preprocess, 176.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
t

[('\nEXPERIENCE PROFESSIONNELLE\n2021 (3 mois)\nJournal Al Chourouk; journaliste Sportil.\n2017-2021\nNessma TV, journaliste\n\'Nessma Sport" .\n2016-2017\nSuperviseur sur Terrain, Societe One To One\nsociete de statistique et sondage dopinionn.\n2016 (Smois)\nAnimateur\nProducteur , Radio SABRA FM;\nAnimation Radiophonique; Kairaoune\n2014-2015 =\nAnimateur\nProducteur, Radio MED FM;\nAnimation Radiophonique\nNabeul\n2013-2014 -Technicien Support, NZSP\nService informatiquer\nTunis.\n2011-2013 - Enqueteur et Superviseur,\nElka Consulting,\n2008-2011\nEnqueteur;\nInstitut |\'Amouri.',
  'Experience'),
 ('\nCOMPETENCES INFORMATIQUE\nOilice\nU Mark\nPhotoshop\nAdobe premier\nOfticc\nDes auires Iogiciels\nBack',
  'Skills'),
 ('\nFORMATIONS ET DIPLÔMES\nCertilicat , Formation-Praduction " Journalisme Sportil TV du\n15 au 19 avril 2019, CAPJC\n2015\nCenificat Les bases de V\'animation Radiophonique trouver\nposer 5a vOLx\nantenne, Alternative Productions,\nCommunication; Conseil .\n2015\nC

# Nougat

In [ ]:
pip install nougat-ocr

  Obtaining dependency information for nougat-ocr from https://files.pythonhosted.org/packages/81/05/accfaf31a1a6e50f379f0504d374d563707bc62190501f6f0cbad815be3f/nougat_ocr-0.1.17-py3-none-any.whl.metadata
  Obtaining dependency information for timm==0.5.4 from https://files.pythonhosted.org/packages/49/65/a83208746dc9c0d70feff7874b49780ff110810feb528df4b0ecadcbee60/timm-0.5.4-py3-none-any.whl.metadata
  Obtaining dependency information for datasets[vision] from https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for lightning<2022,>=2.0.0 from https://files.pythonhosted.org/packages/49/03/1325c91ec4824ede1102dd1ee240bceb8c5aa2b75c5bec83c59f0bb8da07/lightning-2.2.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/53.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/53.4 kB ? eta -:--:--
     -------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured 0.12.6 requires rapidfuzz==3.6.1, but you have rapidfuzz 3.8.1 which is incompatible.


In [ ]:
!nougat D:\9raya\Stage_PFE\ultralytics_crop_1.jpg

^C



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll...
downloading nougat checkpoint version 0.1.0-small to path C:\Users\trabe\.cache\torch\hub\nougat-0.1.0-small


c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/trabe/.conda/envs/myenv/bin')}
  warn(msg)
c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: C:\Users\trabe\.conda\envs\myenv did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)
2024-04-15 14:26:38.572428: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


config.json:   0%|          | 0.00/557 [00:00<?, ?b/s]
config.json: 100%|██████████| 557/557 [00:00<00:00, 3.92Mb/s]

pytorch_model.bin:   0%|          | 0.00/956M [00:00<?, ?b/s]
pytorch_m


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll...
downloading nougat checkpoint version 0.1.0-small to path C:\Users\trabe\.cache\torch\hub\nougat-0.1.0-small


c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/trabe/.conda/envs/myenv/bin')}
  warn(msg)
c:\Users\trabe\.conda\envs\myenv\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: C:\Users\trabe\.conda\envs\myenv did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)
2024-04-15 14:23:09.893589: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


config.json:   0%|          | 0.00/557 [00:00<?, ?b/s]
config.json: 100%|██████████| 557/557 [00:00<?, ?b/s] 

pytorch_model.bin:   0%|          | 0.00/956M [00:00<?, ?b/s]
pytorch_model.bi

In [ ]:
!pip install PyMuPDF

In [ ]:
from paddleocr import PaddleOCR,draw_ocr
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(lang='en') # need to run only once to download and load model into memory

[2024/04/18 11:42:47] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, shape_info_filename=None, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\trabe/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\trabe/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_batch_nu

In [ ]:
img_path = "ultralytics_crop_0.jpg"


In [ ]:
result = ocr.ocr(img_path, cls=False)

: 

In [ ]:
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

: 

In [ ]:
!paddleocr --image_dir ultralytics_crop_0.jpg --use_angle_cls true --lang en

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\trabe\AppData\Roaming\Python\Python39\Scripts\paddleocr.exe\__main__.py", line 4, in <module>
  File "C:\Users\trabe\AppData\Roaming\Python\Python39\site-packages\paddleocr\__init__.py", line 14, in <module>
    from .paddleocr import *
  File "C:\Users\trabe\AppData\Roaming\Python\Python39\site-packages\paddleocr\paddleocr.py", line 48, in <module>
    from tools.infer import predict_system
  File "C:\Users\trabe\AppData\Roaming\Python\Python39\site-packages\paddleocr\tools\infer\predict_system.py", line 32, in <module>
    import tools.infer.predict_rec as predict_rec
  File "C:\Users\trabe\AppData\Roaming\Python\Python39\site-packages\paddleocr\tools\infer\predict_rec.py", line 31, in <module>
    from ppocr.p